In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import emcee
import batman
import sys

In [ ]:
# you need to have exonailer installed
# append to sys.path the directory where you find the file 'ajplanet.so'
sys.path.append('..')
from utilities.ajplanet import pl_rv_array as rv_curve

In [ ]:
# fix the eccentricity (to zero)
fix_ecc = True

In [ ]:
class Planet(object):
    def __init__(self):
        # these are fixed
        self.limb_dark = "quadratic" # limb darkening model
        self.u = [.4983, 0.2042]     # limb darkening coefficients
        
        # transit parameters
        self.rp = 0.          # planet radius (in units of stellar radii)
        self.a = 0.           # semi-major axis (in units of stellar radii)
        self.inc = 0.         # orbital inclination (in degrees)
        self.t0 = 0.          # time of inferior conjunction
        self.jitter_lc = 0.

        # rv parameters
        self.rvsys = 0.      # radial velocity systematic velocity
        self.K = 0.          # semi-amplitude
        self.T0 = 0.         # time of periastron (function of self.t0, not a free parameter)
        self.jitter_rv = 0.

        # parameters shared between transit and rv
        self.period = 0. # orbital period
        self.ecc = 0.    # eccentricity
        self.w = 0.      # longitude of periastron (in degrees)

        
        # number of free parameters        
        self.N_free_parameters = 


        # start the batman model
        self.params = batman.TransitParams()
        self.params.limb_dark = self.limb_dark  # limb darkening model
        self.params.u = self.u                  # limb darkening coefficients

        # names of the parameters for the plots
        self.labels = [r'$R_p / R_*$',
                       r'$a/R_*$',
                       r'$i$',
                       r'$t_0$',
                       r'$s_{\rm lc}$',
                       'vsys',
                       r'$K$',
                       r'$s_{\rm RV}$',
                       r'$P$',
                       r'$e$',
                       r'$\omega$']
        
        if fix_ecc:
            self.labels.pop(10)
            self.labels.pop(9)


    def set_rv_parameters(self, pars):
        """ pars should be [rvsys, K, jitter_rv] """
        self.rvsys, self.K, self.jitter_rv = pars


    def set_transit_parameters(self, pars):
        """ pars should be [rp, a, inc, t0, jitter_lc] """
        self.rp, self.a, self.inc, self.t0, self.jitter_lc = pars

        
    def set_shared_parameters(self, pars):
        """ pars should be [period, (ecc, w)] """
        if fix_ecc:
            self.period = pars
            self.ecc = 0.
            self.w = 90.
        else:
            self.period, self.ecc, self.w = pars
        
        self.T0 = self.t0 # set the time of periastron
        

    def get_rv_curve(self, t, debug=False):
        """ return the RV curve evaluated at times t """
        pass

    
    def get_transit_curve(self, t, debug=False):
        """ return the transit curve at times t"""
        
        # set self.params for batman
        self.params.t0 = self.t0
        self.params.per = self.period
        self.params.rp = self.rp
        self.params.a = self.a
        self.params.inc = self.inc
        self.params.ecc = self.ecc
        self.params.w = self.w

        self.batman_model = batman.TransitModel(self.params, t)
        light_curve = self.batman_model.light_curve(self.params)
        
        return light_curve


    def set_priors(self):
        """ Define each parameter prior distribution """
        
        # use the scipy distributions!
        self.prior_rp = stats.                         # planet radius (in units of stellar radii)
        self.prior_a =                                 # semi-major axis (in units of stellar radii)
        self.prior_inc =                               # orbital inclination (in degrees)
        self.prior_t0 =                                # time of inferior conjunction
        self.prior_jitter_lc = 

        # rv parameters
        self.prior_rvsys =                             # systematic radial velocity
        self.prior_K =                                 # semi-amplitude
        self.prior_jitter_rv = 
 
        # parameters shared between transit and rv
        self.prior_period =                            # orbital period
        
        if not fix_ecc:
            self.prior_ecc =                      # eccentricity
            self.prior_w =                        # longitude of periastron (in degrees)


    def get_from_prior(self, nwalkers):
        """ return a list with random values from each parameter's prior """       
        self.set_priors()

        pars_from_prior = []
        for i in range(nwalkers):
            # use again the scipy distributions
            random_rp = ...
            ...
            
            pars_from_prior.append([random_rp,random_a, random_inc, random_t0, random_jitter_lc,
                                    random_rvsys, random_K, random_jitter_rv,
                                    random_period])

            if not fix_ecc:
                random_ecc = self.prior_ecc.rvs()
                random_w = self.prior_w.rvs()
                
                pars_from_prior.append(random_ecc, random_w)

        return pars_from_prior

In [56]:
### A very simple Data class to hold the light curve and the RVs
class Data(object):
    def __init__(self, rv_file, lc_file, skip_rv_rows=2, skip_lc_rows=0):
        
        self.rv_file = rv_file
        self.lc_file = lc_file

        # read RVs
        self.RVtime, self.RV, self.RVerror = np.loadtxt(rv_file, 
                                                        unpack=True, skiprows=skip_rv_rows)

        # read light curve
        self.LCtime, self.LC, self.LCerror = np.loadtxt(lc_file,
                                                        unpack=True, skiprows=skip_lc_rows)

        self.N_rvs = self.RVtime.size
        self.N_lc = self.LCtime.size

In [ ]:
### This is the log likelihood function

def lnlike(pars, planet, data, debug=False):
    """ pars should be
    # [rp, a, inc, t0, jitter_lc,
    #  rvsys, K, jitter_rv,
    #  period, ecc, w]
    """
    log2pi = np.log(2*np.pi)


    # set the transit params
    planet.set_transit_parameters(pars[:5])
    # set the RV params
    planet.set_rv_parameters(pars[5:8])
    # set the shared params
    planet.set_shared_parameters(pars[8:])

    
    # calculate the lnlike for transit (eq. 3)
    log_like_transit = 
    
    # if you want, try to calculate lnlike using one of the scipy distributions
    # log_like_transit2 = 
    
    
    
    # calculate the lnlike for RVs
    log_like_rv =


    
    # the total log likelighood 
    log_like = 
    
    
    if not np.isfinite(log_like):
        return -np.inf
    else:
        return log_like

In [ ]:
### This function calculate the log prior for a set of parameters
def lnprior(pars, planet, data, debug=False):
    """ pars should be
    # [rp, a, inc, t0, jitter_lc,
    #  rvsys, K, jitter_rv,
    #  period, ecc, w]
    """

    # transit parameters
    prior_rp = 
    prior_a = 
    prior_inc = 
    prior_t0 = 
    prior_jitter_lc =
    
    # rv parameters
    prior_rvsys = 
    prior_K = 
    prior_jitter_rv = 

    # parameters shared between transit and rv
    prior_period =

    
    # total log prior
    ln_prior = 

    
    if not fix_ecc:
        prior_ecc = 
        prior_w = 

        # change ln_prior

    return ln_prior

In [ ]:
### posterior distribution
def lnprob(pars, planet, data, debug=True):
    pass

In [ ]:
### initialize the Planet and Data classes
planet = Planet()
data = Data(rv_file='../EPIC-9792_SOPHIE.rdb', lc_file='../cuttransits.txt')

In [ ]:
### parameters for emcee
ndim, nwalkers = 

In [ ]:
# get random starting positions from the priors
pos = planet.get_from_prior(nwalkers)

In [ ]:
# sample the posterior
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(planet, data))

nsteps = 50
out = sampler.run_mcmc(pos, nsteps)

In [ ]:
## optionally remove some of the initial steps
burnin = 0
samples = sampler.chain[:, burnin:, :].reshape((-1, ndim))

In [ ]:
## make a corner plot of the MCMC samples
import corner

...

In [ ]:
## get the medians of the posterior distributions
median_pars = 


print ['%8s' % s.replace('$', '').replace('_', '').replace('\\rm', '').replace('\\','') for s in planet.labels]
print ['%8.4f' % s for s in median_pars]

In [ ]:
# set the transit params
planet.set_transit_parameters(median_pars[:5])
# set the RV params
planet.set_rv_parameters(median_pars[5:8])
# set the shared params
planet.set_shared_parameters(median_pars[8:])

In [ ]:
fig = plt.figure()

#
time = np.linspace(data.LCtime.min(), data.LCtime.max(), 5000)
lc = planet.get_transit_curve(time)

ax = fig.add_subplot(311)
ax.plot(time, lc, lw=2)
ax.plot(data.LCtime, data.LC, 'o', ms=2)

#
phase0 = (data.LCtime - planet.t0) / planet.period
phase = phase0 % 1
phase[np.where(phase>0.5)[0]]-=1

ax = fig.add_subplot(312)
ax.plot(phase, data.LC, 'o')

#
phase = ((time - planet.t0) / planet.period) % 1
phase[np.where(phase>0.5)[0]]-=1
LCfold = lc[np.argsort(phase)]

ax.plot(np.sort(phase), LCfold, '-')
ax.set_xlim([-0.1, 0.1])

#
time = np.linspace(data.RVtime.min(), data.RVtime.max(), 1000)
rv = planet.get_rv_curve(time)

ax = fig.add_subplot(313)
ax.errorbar(data.RVtime, data.RV - median_pars[5], data.RVerror, fmt='o')
ax.plot(time, rv*1e-3, 'k', alpha=0.7)


plt.show()